# TFM - Visualización de los Datos
## Ismael Franco Hernando

### Imports

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_plotly_dash import JupyterDash
from mysql.connector import (connection)

### Conexión con la base de datos

In [2]:
# Se realiza la conexión con la base de datos
try:
    cnx = connection.MySQLConnection(user='leer', 
                                     password='candanal',
                                     host='138.100.82.178',
                                     database='deepquality')

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Usuario o contraseña incorrecto")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe")
    else:
        print(err)

### Cargamos los datos

In [3]:
# Guardamos y mostramos las bobinas clasificadas
q01 = "SELECT * FROM V_Coils"
clasif = pd.read_sql(q01,cnx)
clasif

,id,SID,PLANT,NAME,STARTTIME,MATERIAL,LENGTH,WIDTH,THICK,WEIGHT,BASEPID,CLASSLABEL,Label,ZnMin,ZnMax
0,1161,226031456,1066.0,226031456,2022-01-28,0.0,792.0,1183.0,3.0,22.0,220100.0,NOK,0,60,80
1,1162,226101621,1066.0,226101621,2022-01-31,0.0,273.0,1252.0,4.0,10.0,220100.0,NOK,2,215,258
2,1163,226120037,1066.0,226120037,2022-01-31,0.0,599.0,1305.0,2.0,15.0,220100.0,OK,1,155,175
3,1164,225387481,1066.0,225387481,2022-01-13,0.0,268.0,1502.0,5.0,15.0,220100.0,OK,1,60,80
4,1165,225406471,1066.0,225406471,2022-01-13,0.0,735.0,1158.0,2.0,19.0,220100.0,NOK,0,45,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,2316,233642421,1066.0,233642421,2022-08-31,0.0,576.0,1422.0,3.0,24.0,220800.0,OK,2,138,158
1156,2317,233445180,1066.0,233445180,2022-08-25,0.0,513.0,1437.0,4.0,23.0,220800.0,OK,1,125,145
1157,2318,232624927,1066.0,232624927,2022-08-01,0.0,345.0,1482.0,2.0,11.0,220800.0,OK,2,155,175
1158,2319,232635130,1066.0,232635130,2022-08-01,0.0,457.0,968.0,3.0,11.0,220800.0,OK,1,88,108


In [4]:
# Guardamos y mostramos los valores correspondientes a las imágenes 1D
q02 = "SELECT * FROM GR_LPValues"
unoD = pd.read_sql(q02,cnx)
unoD = unoD.sort_values(['COILID', 'MID', 'TILEID']).reset_index(drop=True)
unoD

,COILID,TILEID,MID,PLANT,SIDE,MEAN,MAX,MIN,COUNT
0,225216688,0.0,123.0,1066.0,T,88.94490,88.94490,88.94490,1.0
1,225216688,5120.0,123.0,1066.0,T,81.65457,81.65457,81.65457,1.0
2,225216688,10240.0,123.0,1066.0,T,94.79793,94.79793,94.79793,1.0
3,225216688,15360.0,123.0,1066.0,T,127.82180,127.82180,127.82180,1.0
4,225216688,20480.0,123.0,1066.0,T,125.88000,125.88000,125.88000,1.0
...,...,...,...,...,...,...,...,...,...
115967,233667056,109056.0,202.0,1066.0,T,51.92412,51.92412,51.92412,1.0
115968,233667056,114176.0,202.0,1066.0,T,52.02400,52.02400,52.02400,1.0
115969,233667056,119552.0,202.0,1066.0,T,48.61295,48.61295,48.61295,1.0
115970,233667056,124672.0,202.0,1066.0,T,42.18800,42.18800,42.18800,1.0


In [5]:
# Se cierra la conexión con la base de datos
cnx.close()

### Visualización de las bobinas - 1D

In [6]:
# Se obtienen las IDs únicas de las bobinas
labels = unoD['COILID'].unique()

In [7]:
print("Seleccione la ID de la bobina a mostrar:")
# Se crea la app
app = JupyterDash('ZN-1D')
# Se añaden el menú desplegable con las IDs de las bobinas y el espacio donde se añadirán
# los diefrentes gráficos
app.layout = html.Div([
    dcc.Dropdown( id='dropdown', options=[{'label': i, 'value': i} for i in labels], value = labels[0]),
    html.Div(dcc.Graph(id='dd-output-container'))
])


# Cada vez que se seleccione una ID se actualiza el displey
@app.callback(
    Output('dd-output-container', 'figure'),
    Input('dropdown', 'value')
)
# Función que genera los gráficos para cada ID
def update_output(value):
    bobina = value
    # Se crea una figura de 2 x 2 (un gráfico para cada sensor)
    fig = make_subplots(rows=2, cols=2, subplot_titles=('Cara de Arriba - Sensor 123',
                                                        'Cara de Abajo - Sensor 124',
                                                        'Cara de Arriba - Sensor 201',
                                                        'Cara de Abajo - Sensor 202'))
    
    # Se obtienen los valores máximos y mínimos de Zn aceptados en la bobina
    maxZn = clasif.loc[clasif['SID']==bobina].ZnMax.iloc[0]
    minZn = clasif.loc[clasif['SID']==bobina].ZnMin.iloc[0]
    
    # Se obtienen los valores medidos por el sensor 123
    arriba1 = unoD.loc[unoD['COILID']==bobina].loc[unoD['MID']==123.0]
    # El eje x serán las ID de las tejas de la bobina
    x = arriba1['TILEID'].tolist()
    # El eje y serán el valor medio de Zn de cada teja
    y = arriba1['MEAN'].tolist()
    
    # Se añade el scatter con los valores obtenidos en los pasos anteriores
    fig.add_trace(
        go.Scatter(x=x, y=y),
        row=1, col=1
    )
    
    # Se obtienen los valores medidos por el sensor 124
    arriba2 = unoD.loc[unoD['COILID']==bobina].loc[unoD['MID']==124.0]
    x2 = arriba2['TILEID'].tolist()
    y2 = arriba2['MEAN'].tolist()

    fig.add_trace(
        go.Scatter(x=x2, y=y2),
        row=1, col=2
    )
    
    # Se obtienen los valores medidos por el sensor 201
    abajo1 = unoD.loc[unoD['COILID']==bobina].loc[unoD['MID']==201.0]
    x3 = abajo1['TILEID'].tolist()
    y3 = abajo1['MEAN'].tolist()

    fig.add_trace(
        go.Scatter(x=x2, y=y2),
        row=2, col=1
    )
    
    # Se obtienen los valores medidos por el sensor 202
    abajo2 = unoD.loc[unoD['COILID']==bobina].loc[unoD['MID']==202.0]
    x4 = abajo2['TILEID'].tolist()
    y4 = abajo2['MEAN'].tolist()

    fig.add_trace(
        go.Scatter(x=x2, y=y2),
        row=2, col=2
    )

    # Se configura el tamaño de la figura y se añade el título
    fig.update_layout(height=540, width=790, title_text="Bobina " + str(bobina))
    
    # Para cada gráfico se añade el valor máxmimo o mínimo de Zn en forma de recta
    fig['layout'].update(shapes=[{'type': 'line','y0':maxZn,'y1': maxZn,'x0':str(0), 
                                  'x1':str(max(x)),'xref':'x1','yref':'y1',
                                  'line': {'color': 'green','width': 2}},
                                 {'type': 'line','y0':minZn,'y1': minZn,'x0':str(0), 
                                  'x1':str(max(x)),'xref':'x1','yref':'y1',
                                  'line': {'color': 'green','width': 2}},
                                 {'type': 'line','y0':maxZn,'y1': maxZn,'x0':str(0), 
                                  'x1':str(max(x2)),'xref':'x2','yref':'y2',
                                  'line': {'color': 'green','width': 2}},
                                 {'type': 'line','y0':minZn,'y1': minZn,'x0':str(0), 
                                  'x1':str(max(x)),'xref':'x2','yref':'y2',
                                  'line': {'color': 'green','width': 2}},
                                 {'type': 'line','y0':maxZn,'y1': maxZn,'x0':str(0), 
                                  'x1':str(max(x)),'xref':'x3','yref':'y3',
                                  'line': {'color': 'green','width': 2}},
                                {'type': 'line','y0':minZn,'y1': minZn,'x0':str(0), 
                                  'x1':str(max(x3)),'xref':'x3','yref':'y3',
                                  'line': {'color': 'green','width': 2}},
                                 {'type': 'line','y0':maxZn,'y1': maxZn,'x0':str(0), 
                                  'x1':str(max(x)),'xref':'x4','yref':'y4',
                                  'line': {'color': 'green','width': 2}},
                                {'type': 'line','y0':minZn,'y1': minZn,'x0':str(0), 
                                  'x1':str(max(x4)),'xref':'x4','yref':'y4',
                                  'line': {'color': 'green','width': 2}}])
    
    # Se devuelve la figura obtenida
    return fig

# Se muestra la aplicación
app

Seleccione la ID de la bobina a mostrar:
